# Librerías

In [1]:
import numpy as np
import pandas as pd

# Lectura de datos

In [2]:
data = pd.read_csv('./datasets/tweets.csv', sep=';', on_bad_lines='skip',
                   lineterminator='\n', low_memory=False)
data

,id,user,fullname,url,timestamp,replies,likes,retweets,text\r
0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0,0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,1.132977e+18,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0,Cardano: Digitize Currencies; EOS https://t.co...
2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1,Another Test tweet that wasn't caught in the s...
3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...
...,...,...,...,...,...,...,...,...,...
16889760,1.198262e+18,JacobCanfield,Jacob Canfield,NaN,2019-11-23 15:28:50+00,2,16,3,Happy #FibonacciDay \n\nA while back I created...
16889761,1.198266e+18,Vizique,Vizique,NaN,2019-11-23 15:45:55+00,0,0,0,Bitcoin Suisse Certificates :) https://t.co/nd...
16889762,1.198266e+18,torusJKL,Gal Buki ($torusJKL),NaN,2019-11-23 15:45:56+00,0,0,0,Register now for the early access of the Codug...
16889763,1.198266e+18,Adekunl95628158,Adekunle Daniel,NaN,2019-11-23 15:45:57+00,0,0,0,@btc \n@btc \nDo you know that BTC Baskets isn...


In [3]:
data = data[['timestamp', 'text\r']]
data = data.rename(columns={'timestamp': 'date', 'text\r': 'text'})
data

,date,text
0,2019-05-27 11:49:14+00,È appena uscito un nuovo video! LES CRYPTOMONN...
1,2019-05-27 11:49:18+00,Cardano: Digitize Currencies; EOS https://t.co...
2,2019-05-27 11:49:06+00,Another Test tweet that wasn't caught in the s...
3,2019-05-27 11:49:22+00,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,2019-05-27 11:49:23+00,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...
...,...,...
16889760,2019-11-23 15:28:50+00,Happy #FibonacciDay \n\nA while back I created...
16889761,2019-11-23 15:45:55+00,Bitcoin Suisse Certificates :) https://t.co/nd...
16889762,2019-11-23 15:45:56+00,Register now for the early access of the Codug...
16889763,2019-11-23 15:45:57+00,@btc \n@btc \nDo you know that BTC Baskets isn...


# Limpieza de datos

In [4]:
data.isna().sum()

date    0
text    0
dtype: int64

In [5]:
from datetime import datetime

def str_to_datetime(d):
    return datetime.strptime(d.split(' ')[0], '%Y-%m-%d')

data.date = np.vectorize(str_to_datetime)(np.array(data.date))
data = data.sort_values(by='date')
data = data.reset_index(drop=True)

In [6]:
from datetime import timedelta

current_day = data.date.min()
min_index = 0
count_tweets = 0

# Verificar que hay mínimo 1500 publicaciones por día
for t in data.itertuples():
    index = t[0]
    d = t[1]

    if current_day == d:
        count_tweets += 1
        continue

    # ASD ASD ASD ASD ASD
    if (count_tweets < 2000):
        min_index = index + 1

    current_day += timedelta(days=1)
    count_tweets = 1

data = data.loc[min_index:]
data.reset_index(drop=True, inplace=True)

In [7]:
# from cmath import nan

# # Quedarse solo con n publicaciones por dia
# def keep_n_tweets_per_day(n):
#     current_date = data.date.min()
#     arr_days = []
#     count_tweets = 0

#     for t in data.itertuples():
#         index = t[0]
#         d = t[1]

#         if current_date == d:
#             if count_tweets == n:
#                 arr_days.append(nan)
#                 continue
#             count_tweets += 1
#         else:
#             current_date += timedelta(days=1)
#             count_tweets = 1

#         arr_days.append(d)

#     data.date = arr_days
#     data.dropna(inplace=True)
#     data.reset_index(drop=True, inplace=True)

In [7]:
from cmath import nan
from random import sample, seed

# Quedarse solo con n publicaciones por dia
def keep_n_tweets_per_day(n):
    seed(12)
    current_date = data.date.min()
    days = []
    a = []

    for t in data.itertuples():
        index = t[0]
        d = t[1]

        if current_date == d:
            a.append(index)
        else:
            current_date += timedelta(days=1)
            try:
                days.extend(sample(a, n))
            except:
                days.extend(a)
            a = [index]

    days.extend(sample(a, n))
    days.sort()

    arr_days = []
    i = 0

    for d in data.itertuples():
        try:
            if d[0] != days[i]:
                arr_days.append(None)
            else:
                arr_days.append(d[1])
                i += 1
        except:
            arr_days.append(None)

    data.date = arr_days
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)

In [8]:
# keep_n_tweets_per_day(1500)
keep_n_tweets_per_day(2000)

In [9]:
import re
from langdetect import detect

english_texts = []

for d in data.itertuples():
    t = d[2]

    try:
        t = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', t, flags=re.MULTILINE)
        t = re.sub(r'\$\w+[,]|\@\w+|[,]\@\w+', '', t)

        t = t.replace('#', '').replace('\n', '').replace('\r', '').replace('\b', '').replace(
            '\t', '').replace('\\', ' ').replace('/', ' ').replace('|', ' ')

        t = re.sub(r'  +', ' ', t)

        if detect(t) != 'en':
            english_texts.append(None)
        else:
            english_texts.append(t)
    except:
        english_texts.append(None)

In [10]:
data.text = english_texts
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

In [11]:
keep_n_tweets_per_day(1000)

In [15]:
def check_skipped_days():
    x = 0
    arr = np.array([])
    day_before = datetime.now()
    index_before = 0
    min_index = 0

    for t in data.itertuples():
        index = t[0]
        d = t[1]
        diff = 0

        if x == 0:
            day_before, index_before = d, index
            x+=1
            continue

        diff = (d - day_before).days

        if diff > 1:
            arr = np.append(arr, '{} => {} : {} days'.format(index_before, index, diff))
            min_index = index_before

        day_before, index_before = d, index

    return arr, min_index + 1

In [16]:
skipped_days, min_index = check_skipped_days()

print(skipped_days)
print(len(skipped_days))

[]
0


In [17]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def analyze_sentiments():
    a = 0
    df = pd.DataFrame()
    sentiments = []

    sia = SentimentIntensityAnalyzer()

    for t in data.text:
        x = sia.polarity_scores(t)

        sentiments.append(x['compound'])

    return sentiments

In [18]:
data['sentiment'] = analyze_sentiments()

In [19]:
data

,date,text,sentiment
0,2019-05-10,More colleges are offering classes in blockcha...,0.0000
1,2019-05-10,"whale_alert: 1,000 BTC (6,387,933 USD) transfe...",0.0000
2,2019-05-10,"Liquidated BITMEX short on XBTUSD:📉 6,362.50 $...",0.0000
3,2019-05-10,Bull flag has confirmed in the Bitcoin market ...,0.2023
4,2019-05-10,Scary story : what's a bubble?this chart shows...,-0.5204
...,...,...,...
197995,2019-11-23,AbhinavEspsoft.: buy bitcoin using fiat curren...,-0.2732
197996,2019-11-23,It is not that the best thing to do was to no...,0.8159
197997,2019-11-23,🔥🔥15m Volume Alert!🔥🔥 $MCO current volume: 10....,-0.6476
197998,2019-11-23,107 BTC (417) deposited from unknown address (...,0.0000


In [20]:
data = data[['date', 'sentiment']]

In [21]:
data.to_csv('./consolidated_dataset/tweets.csv', index=False)